# PIPELINE

In [154]:
from sentence_transformers import SentenceTransformer, util
from utils.operate_data import load_dataset as dd_load_dataset
from utils.autocomplete import yolo_utils as yolo
import pandas as pd

from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM

from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import train_test_split
import numpy as np



from tqdm import tqdm
tqdm.pandas()

%matplotlib inline

## get data

In [128]:
df_meta, _ = dd_load_dataset('/local_data/meta_data/', df_name = 'meta.pickle', files_df_name=None)
df_meta.reset_index(inplace=True)

df_prepared = pd.read_pickle('/local_data/meta_data/prepared_files.pickle')
df_prepared = df_prepared[df_prepared.extension == '.pdf']
df_prepared = df_prepared[df_prepared.content.agg(len) != 0]
df_prepared = df_prepared.loc[df_prepared.groupby('doc_id')['order'].idxmax()]
df_prepared.rename(columns={'content': 'text'}, inplace=True)

df_content = df_prepared.merge(df_meta, on='doc_id')
df_content = df_content[['doc_id', 'text', 'content']]

In [129]:
# for research
# df_content = df_content.head(100)

In [130]:
# model_sentence = SentenceTransformer('bert-base-nli-mean-tokens')
# model_sentence.save('/local_data/models/text_matching/')

In [131]:
model_summarization = AutoModelForSeq2SeqLM.from_pretrained( '/local_data/summarization/models/csebuetnlp_mT5_m2o_russian_crossSum/')
tokenizer = AutoTokenizer.from_pretrained('/local_data/summarization/data/csebuetnlp_mT5_m2o_russian_crossSum')

# get summarization

In [132]:
def summarizer(text: str, tokenizer: AutoTokenizer, model = AutoModelForSeq2SeqLM) -> str:
    
    inputs = tokenizer(text, max_length=1024, truncation=True, return_tensors="pt").input_ids
    outputs = model.generate(inputs, max_new_tokens=200, do_sample=False)
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [6]:
# при необходимости получить суммаризацию. для экспериментов достаточно пользоваться фичёй content

# df_content['predict_summary'] = df_content['text'].progress_apply(lambda row: summarizer(row, tokenizer, model_summarization))

# get embeddings with sentence transformer

In [53]:
%%time
corpus = df_content['content'].to_list()
corpus_embeddings = model_sentence.encode(corpus, convert_to_tensor=True)

CPU times: user 1h 3min 34s, sys: 2min 23s, total: 1h 5min 58s
Wall time: 4min 11s


# Cross Encoding

In [22]:
from sentence_transformers.cross_encoder import CrossEncoder

In [ ]:
num_candidates = 500
max_corpus_size = 20000

In [28]:
cross_encoder_model = CrossEncoder('cross-encoder/stsb-distilroberta-base')

In [88]:
# example
# sentence_pairs = [['О согласовании проекта внесения изменений в генеральный план', 'О вопросах согласования проекта внесения изменений в генеральный план']]
# ce_scores = cross_encoder_model.predict(sentence_pairs)
# ce_scores

array([    0.96888], dtype=float32)

In [83]:
%%time
query = df_content.iloc[88].content
sentence_combinations = [[query, sentence] for sentence in corpus[:1000]]

similarity_scores = cross_encoder_model.predict(sentence_combinations)

CPU times: user 22min 53s, sys: 4min 55s, total: 27min 49s
Wall time: 1min 59s


In [84]:
sim_scores_argsort = reversed(np.argsort(similarity_scores))

In [86]:
# Print the scores
print("Query:", query)
c = 0
for idx in sim_scores_argsort:
    print("{:.2f}\t{}".format(similarity_scores[idx], corpus[idx]))
    c += 1
    if c >= 10:
        break
# similarity_scores[0], corpus[0]

Query: О согласовании проекта внесения изменений в генеральный план
0.92	О вопросах согласования проекта генерального плана
0.92	О вопросах согласования проекта генерального плана
0.91	О вопросах согласования проектов схем территориального планирования
0.91	О вопросах согласования проектов схем территориального планирования
0.91	О согласовании проекта федерального закона «О внесении изменений в Федеральный закон «Об использовании атомной энергии» в части регулирования безопасности деятельности по эксплуатации радиационны х источников и оценки соответствия оборудования, изделий и технологий (процессов) для объектов использования атомной энергии»
0.91	О согласовании проекта Правительства Российской Федерации "О внесении изменений в Положение об управлении находящимися в федеральной собственности акциями акционерных обществ и использовании специального права на участие Российской Федерации в управлении акционерными обществами ("золотой акции")

0.91	О законопроектах, направленных на совер

In [56]:
len(corpus[1:100]), len(similarity_scores)

(99, 99)

# Community detection

In [89]:
# %%time
# clusters = util.community_detection(corpus_embeddings, min_community_size=25, threshold=0.5)

In [14]:
clusters[0][-1]

8057

In [15]:
#Print for all clusters the top 3 and bottom 3 elements
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    # for sentence_id in cluster[0:3]:
    #     print("\t", corpus[sentence_id])
    # print("\t", "...")
    # for sentence_id in cluster[-3:]:
    #     print("\t", corpus[sentence_id])
    # break


Cluster 1, #8057 Elements 


# DBSCAN

In [52]:
from sklearn.cluster import DBSCAN

In [93]:
db = DBSCAN(eps=0.05, min_samples=2, leaf_size=2).fit(corpus_embeddings)

In [94]:
db.labels_

array([ -1,  -1,   0, ...,  -1, 367,  -1])

In [95]:
df_content['cluster'] = db.fit_predict(corpus_embeddings)

In [96]:
df_content.cluster.value_counts()

cluster
-1      6631
 8        26
 7        24
 33       22
 56       21
        ... 
 205       2
 204       2
 203       2
 202       2
 445       2
Name: count, Length: 447, dtype: int64

# Aglomerative Clustering to set the number of clusters

In [79]:
%%time
# изменить фичу content на predict_summary при необходимости

# Normalize the embeddings to unit length
np_corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

CPU times: user 6.85 ms, sys: 0 ns, total: 6.85 ms
Wall time: 5.75 ms


In [7]:
# 0.05

In [8]:
%%time
# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=0.04,  
                                           affinity = 'cosine', linkage ='average')
clustering_model.fit(np_corpus_embeddings)
cluster_assignment = clustering_model.labels_

CPU times: user 6.25 s, sys: 79.9 ms, total: 6.33 s
Wall time: 6.33 s


In [9]:
clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])

In [10]:
df_content['cluster'] = clustering_model.fit_predict(np_corpus_embeddings)

In [11]:
print(f'количество документов: {df_content.shape[0]},\nколичество кластеров: {len(clustered_sentences.items())}')

количество документов: 8058,
количество кластеров: 1084


In [12]:
df_content.iloc[476]['cluster']

5

In [13]:
# cluster_assignment[7984]

In [111]:
# df_content[df_content.cluster == 161]

## Set only big clusters

In [145]:
corpus_embeddings.shape, cluster_assignment.shape

(torch.Size([8058, 768]), (8058,))

In [98]:
corpus_emb= corpus_embeddings[:10000, :]
cluster_assign = cluster_assignment[:10000]

In [99]:
df_clusters = pd.DataFrame(corpus_emb)
features = df_clusters
df_clusters['cluster'] = cluster_assign
df_clusters.columns = df_clusters.columns.map(str)

In [100]:
df_clusters.cluster.nunique()

1084

In [101]:
# создание списка кластеров имеющих менее "x" экземпляров

x = 10
num_clusters = df_clusters.cluster.value_counts().to_dict()

single_clusters = []
for key, val in num_clusters.items():
    if val <= x:
        single_clusters.append(key)
len(single_clusters)

992

In [102]:
# кластеры имеющие количество экземпляров менее 20 объединяются в один кластер
last_cluster = len(clustered_sentences.items()) + 1 # нэйминг последнего объеденённого кластера
df_clusters['cluster'] = df_clusters['cluster'].apply(lambda row: last_cluster  if row in single_clusters else row)

In [103]:
df_clusters.cluster.nunique()

93

In [104]:
target = df_clusters['cluster']

In [105]:
# df_clusters = pd.DataFrame(corpus_emb)
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

In [54]:
# from sklearn.cluster import DBSCAN
# from sklearn.cluster import KMeans
# from sklearn.metrics import mean_squared_error

# Random Forest Classifier for predict clusters

In [106]:
from sklearn.model_selection import GridSearchCV#, HalvingGridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score

In [61]:
# param_grid = { 
#     'n_estimators': [700, 1000],
#     'criterion': ['gini', 'entropy', 'log_loss'],
#     'min_samples_split':[5,10,20]
# }

In [65]:
# CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=3)
# CV_rfc.fit(X_train, y_train)
# print (CV_rfc.best_params_)

In [48]:
# params = CV_rfc.best_params_
rfc = RandomForestClassifier(random_state=42, verbose=1, n_jobs=2)#, **params)
rfc.fit(X_train, y_train)
predicts = rfc.predict(X_test)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    2.1s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    4.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished


In [66]:
params

{'criterion': 'gini', 'min_samples_split': 5, 'n_estimators': 1000}

In [49]:
precision_score(y_test, predicts, average='weighted'), recall_score(y_test, predicts, average='weighted')

/opt/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.8754299235939375, 0.8606286186931348)

In [68]:
predicts

array([   5,    5, 1085, ..., 1085, 1085, 1085])

# CatBoost for predict clusters

In [119]:
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor


from sklearn.metrics import precision_score, recall_score
cat = CatBoostClassifier(random_state=42)

In [122]:
%%time
cat.fit(X_train,y_train, verbose=False, plot=False)

CPU times: user 17h 45min 54s, sys: 49.1 s, total: 17h 46min 43s
Wall time: 46min 41s


In [29]:
# from catboost import Pool, cv
# params = {"iterations": 100,
#           "depth": 2,
#           "loss_function": "RMSE",
#           "verbose": False}
# cv_dataset = Pool(data=X_train,
#                   label=y_train)
# scores = cv(cv_dataset,
#             params,
#             fold_count=2, 
#             plot="True")

In [123]:
cat.predict(X_test)

array([[   5],
       [   5],
       [ 480],
       ...,
       [1085],
       [   5],
       [1085]])

In [124]:
predicts = cat.predict(X_test)

In [125]:
precision_score(y_test, predicts, average='weighted'), recall_score(y_test, predicts, average='weighted')

(0.9890440710432155, 0.9884201819685691)

In [126]:
cat.save_model('/local_data/models/text_matching/catboost_clusters.cbm')

In [112]:
y_test.head(30)

3826       5
4823       5
2034     480
5928      45
3341       5
2107    1085
5086      52
222       77
6706     161
3703     268
2619     103
5602    1085
4857      57
8044       5
5995     103
2929     263
6152       6
6127      77
4262     127
4652    1085
7148    1085
2534       5
1075    1085
5351      38
472       77
4706     137
3482       5
6044       5
7374    1085
5817     103
Name: cluster, dtype: int64

In [113]:
predicts[103]

array([5])

In [114]:
y_test.value_counts()

cluster
1085    667
5       602
161     120
77       90
6        71
       ... 
479       2
432       2
404       2
7         1
258       1
Name: count, Length: 92, dtype: int64

## Kmeans

In [195]:
mdl = KMeans(n_clusters=90)
mdl.fit(X_train)
predicts = mdl.predict(X_test)

/opt/venv/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/opt/venv/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [196]:
precision_score(y_test, predicts, average='weighted'), recall_score(y_test, predicts, average='weighted')

/opt/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.0, 0.0)

# cluster recognition

In [148]:
sample = 88  # 52
query = [df_content.iloc[sample]['content']]
# corpus = df_content['content'].to_list()
df_cluster = df_content[df_content.cluster == df_content.iloc[sample].cluster].reset_index()
# corpus_cluster = df_cluster['content'].to_list()

AttributeError: 'DataFrame' object has no attribute 'cluster'

In [136]:
df_cluster

NameError: name 'df_cluster' is not defined

In [134]:
# query

In [133]:
# corpus_cluster

In [463]:
%%time
embeddings_query = model_sentence.encode(query, convert_to_tensor=True)
embeddings_corpus = model_sentence.encode(corpus_cluster, convert_to_tensor=True)
top = util.semantic_search(embeddings_query, embeddings_corpus, score_function=util.cos_sim, top_k= 5)[0]
matching = {}
for doc in top:
    doc_id = df_cluster.iloc[doc['corpus_id']]['doc_id']
    matching[doc_id] = doc['score']  

CPU times: user 10.3 s, sys: 25.4 ms, total: 10.4 s
Wall time: 658 ms


In [146]:
corpus_embeddings

tensor([[-0.4683,  0.3372,  0.9311,  ...,  0.0400,  0.0779,  0.3649],
        [-0.6288,  0.2307,  0.9192,  ..., -0.1604,  0.4529,  0.5103],
        [-0.5409,  0.2283,  1.0933,  ..., -0.0259,  0.1508,  0.5428],
        ...,
        [-0.6407,  0.1795,  0.9143,  ...,  0.0736,  0.3708,  0.3535],
        [-0.5498,  0.3906,  1.3660,  ..., -0.0556,  0.1707, -0.0072],
        [-0.6638,  0.4813,  0.9552,  ...,  0.0291,  0.1848,  0.5233]])

In [149]:
%%time
embeddings_query = model_sentence.encode(query, convert_to_tensor=True)
top = util.semantic_search(embeddings_query, corpus_embeddings, score_function=util.cos_sim, top_k= 5)[0]
matching = {}
for doc in top:
    doc_id = df_content.iloc[doc['corpus_id']]['doc_id']
    matching[doc_id] = doc['score'] 

CPU times: user 4.35 s, sys: 0 ns, total: 4.35 s
Wall time: 439 ms


In [150]:
matching

{'c95c2c21-8a61-4023-ad8f-25faaf3cf687': 1.000000238418579,
 '1869c7c4-c00d-41f9-b5ee-211dc204eb29': 1.0000001192092896,
 '11439a8e-a2f9-465a-829c-be77a363d065': 1.0000001192092896,
 '02f3049d-6989-44df-85c1-be9d91344b09': 1.0000001192092896,
 '2d8435fe-9c60-449d-af4c-b8549100b93f': 1.0000001192092896}

In [151]:
yolo.show_pdf('c95c2c21-8a61-4023-ad8f-25faaf3cf687')

In [153]:
yolo.show_pdf('11439a8e-a2f9-465a-829c-be77a363d065')

In [514]:
df_content.cluster.value_counts()

cluster
5       1951
161      401
77       336
6        248
127      187
        ... 
1036       1
998        1
713        1
335        1
854        1
Name: count, Length: 1084, dtype: int64

In [125]:
num_clusters = df_content.cluster.value_counts().to_dict()

In [127]:
cl = []
for key, val in num_clusters.items():
    if val == 1:
        cl.append(key)
len(cl)

551

In [458]:
df_content[df_content.cluster == 1004]

,doc_id,text,content,cluster
88,02f3049d-6989-44df-85c1-be9d91344b09,департамент планирования территориального разв...,О согласовании проекта внесения изменений в ге...,1004
97,0327b03d-ec0d-4e5a-abfe-77eded58c00b,департамент планирования россии территориально...,О согласовании проекта внесения изменений в ге...,1004
544,11439a8e-a2f9-465a-829c-be77a363d065,министерство обороны российской федерации мино...,О согласовании проекта внесения изменений в ге...,1004
782,1869c7c4-c00d-41f9-b5ee-211dc204eb29,министерство обороны российской федерации мино...,О согласовании проекта внесения изменений в ге...,1004
925,1db847e3-31f3-4ff5-8177-ed6500e4760b,федеральная служба войск национальной гвардии ...,О вопросах согласования проекта внесения измен...,1004
1410,2d8435fe-9c60-449d-af4c-b8549100b93f,департамент планирования территориального разв...,О согласовании проекта внесения изменений в ге...,1004
1702,36cdf743-5929-4961-a7b3-f6e7cff005e0,федеральная служба войск национальной гвард ро...,О вопросах согласования проекта внесения измен...,1004
1870,3c9f0d3b-8611-4174-a9aa-80cda54a33df,департамент планирования территориального разв...,О согласовании проекта внесения изменений в ге...,1004
2250,4834c6b2-cdb2-409b-ad7c-695b738310b3,министерство департамент планирования россии т...,О согласовании проекта внесения изменений в ге...,1004
2396,4cb92317-c185-48a9-8126-909c1c774497,департамент планирования территориального разв...,О согласовании проекта внесения изменений в ге...,1004


In [155]:
pd.read_pickle('/local_data/summarization/data/df_summary.pickle')

,doc_id,text,kind,summary,len_content,len_text
2,f5805e2c-0ed3-40e4-a16e-5f2370d0412d,"Информируем о том, что среди предприятий-участ...",Письмо,О разъяснении пунктов Соглашения о сотрудничес...,249,573
5,6bab4407-7afd-45bb-9c19-95aeb4055930,В соответствии с письмом Минэкономразвития Рос...,Письмо,Об исполнении поручения Минэкономразвития Росс...,75,542
6,b98b4f3d-08df-4bc2-85a4-90f1af724ad6,Территориальное управление Росимущества в горо...,Письмо,О предоставлении выписок из реестра федерально...,102,495
8,ef76b51a-3f85-4eca-9cb5-20c45f9eb929,В дополнение к письму Аппарата Правительства Р...,Письмо,О позиции Верховного Суда Российской Федерации...,133,325
9,24f62e09-44a5-4284-8ef5-8db6e0d55f0e,В соответствии с поручением Правительства Росс...,Письмо,О предложениях по учету места происхождения эк...,64,2344
...,...,...,...,...,...,...
3424,ed1bd9ae-a134-4e61-b9a3-277aa9e4aa8a,В связи с Вашим запросом направляем статистиче...,Письмо,"О статистике экспорта из Латвии в Россию, Бело...",62,221
3425,60d7f317-622f-489f-9427-e0de2972192a,Федеральное государственное бюджетное образова...,Письмо,"О направлении формы ""Сведения о доходах, получ...",93,417
3427,ddb82548-96c7-4a1d-9ad3-c9c351e9938d,Федеральное агентство по туризму — государстве...,Письмо,О направлении уточненной информации по форме 1...,62,691
3428,f3a088e7-ccbd-4c6a-b5cc-5ca8cde451ed,В рамках актуализации сведений в автоматизиров...,Письмо,Об актуализации сведений в автоматизированной ...,98,803
